# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df.filter(items=['id', 'message', 'original', 'genre'])
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
y['related']=y['related'].map(lambda x: 1 if x == 2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#ML Pipeline
pipeline = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
#split into train and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [9]:
y_test.shape

(6554, 36)

In [13]:
y_train.shape

(19662, 36)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
#run classification reports
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('-----------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      5026
               request       0.85      0.41      0.55      1118
                 offer       0.00      0.00      0.00        31
           aid_related       0.79      0.43      0.55      2691
          medical_help       0.64      0.02      0.03       511
      medical_products       0.50      0.01      0.03       347
     search_and_rescue       0.75      0.02      0.04       163
              security       0.00      0.00      0.00       118
              military       1.00      0.01      0.02       208
                 water       0.89      0.14      0.24       422
                  food       0.85      0.26      0.40       748
               shelter       0.83      0.11      0.20       568
              clothing       0.50      0.01      0.02       113
                 money       0.83      0.03      0.07       145
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
#Test out different params
parameters = {'clf__max_depth': [10, 20, None],
              'clf__min_samples_split': [5, 10],
              'clf__n_estimators': [10, 20]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose=10, n_jobs=-1)
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=10 
[CV]  clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=10, score=0.3919882475187215, total=  10.8s
[CV] clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   17.7s remaining:    0.0s


[CV]  clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=10, score=0.386786355475763, total=  10.7s
[CV] clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   35.3s remaining:    0.0s


[CV]  clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=10, score=0.3882844585344765, total=  10.9s
[CV] clf__max_depth=10, clf__min_samples_split=5, clf__n_estimators=20 


...

[CV]  clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=10, score=0.3816827027418071, total=  10.7s
[CV] clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=20 


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.7min remaining:    0.0s


[CV]  clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=20, score=0.38203545254341303, total=  11.6s
[CV] clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=20 
[CV]  clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=20, score=0.3884184845005741, total=  11.6s
[CV] clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=20 
[CV]  clf__max_depth=10, clf__min_samples_split=10, clf__n_estimators=20, score=0.389209047687653, total=  11.5s
[CV] clf__max_depth=20, clf__min_samples_split=5, clf__n_estimators=10 
[CV]  clf__max_depth=20, clf__min_samples_split=5, clf__n_estimators=10, score=0.42168716924263133, total=  11.1s
[CV] clf__max_depth=20, clf__min_samples_split=5, clf__n_estimators=10 
[CV]  clf__max_depth=20, clf__min_samples_split=5, clf__n_estimators=10, score=0.41662283008942663, total=  11.1s
[CV] clf__max_depth=20, clf__min_samples_split=5, clf__n_estimators=10 
[CV]  clf__max_depth=20, clf__min_samples_split=5, clf__n_estimators

[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 12.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [10, 20, None], 'clf__min_samples_split': [5, 10], 'clf__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('-------------------------------------------------------------------------------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.89      5026
               request       0.84      0.41      0.55      1118
                 offer       0.00      0.00      0.00        31
           aid_related       0.79      0.53      0.63      2691
          medical_help       0.80      0.02      0.05       511
      medical_products       0.75      0.02      0.03       347
     search_and_rescue       0.00      0.00      0.00       163
              security       0.00      0.00      0.00       118
              military       0.71      0.02      0.05       208
                 water       0.93      0.17      0.29       422
                  food       0.88      0.23      0.37       748
               shelter       0.90      0.13      0.22       568
              clothing       0.60      0.03      0.05       113
                 money       1.00      0.03      0.07       145
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
#test best parameter
cv.best_params_

{'clf__max_depth': None, 'clf__min_samples_split': 5, 'clf__n_estimators': 20}

### 9. Export your model as a pickle file

In [12]:
model = pickle.dumps('clf')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.